In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data1 = pd.read_csv('Amazon.csv')
print(data1.shape)
data1.head(5)

(455000, 13)


,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,helpScore,helpful
0,138806,138807,B000E63LME,A1CQGW1AOD0LF2,"Alena K. ""Alena""",1,2,2,1294185600,Not as pictured.,I was looking forward to try cranberry apple f...,0.5,False
1,469680,469681,B004ZIH4KM,A37S7U1OX2MCWI,Becky Cole,0,0,5,1349740800,seeds,"TY for everything. The seeds arrived quickly,...",NaN,False
2,238202,238203,B003ZXE9QA,A2OM6G73E64EQ9,jeff,0,0,5,1329264000,I'm addicted!,I've finally found the best cereal in the worl...,NaN,False
3,485307,485308,B001RVFERK,A25W349EE97NBK,Tangent4,1,1,4,1248307200,I wanted to love these...,I originally bought these chips because I'd he...,1.0,False
4,375283,375284,B000OQZNTS,A3CPPW0HUC07YS,Amy Nicolai,0,0,5,1333238400,Excellent chamomile tea,"Really excellent tea, flowers are visible in t...",NaN,False


In [3]:
data = data1[(data1.Score < 5)]
print(data.shape)
data.head(5)

(164213, 13)


,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,helpScore,helpful
0,138806,138807,B000E63LME,A1CQGW1AOD0LF2,"Alena K. ""Alena""",1,2,2,1294185600,Not as pictured.,I was looking forward to try cranberry apple f...,0.5,False
3,485307,485308,B001RVFERK,A25W349EE97NBK,Tangent4,1,1,4,1248307200,I wanted to love these...,I originally bought these chips because I'd he...,1.0,False
10,201170,201171,B00395570G,A2V2SL3JIOXYVM,doodle,0,0,1,1325203200,What happened 2 my once-favorite coffee?,I have been drinking this Chocolate Raspberry ...,NaN,False
12,105099,105100,B0036VLZDQ,A1WR5TWGNODAHA,Katie R.,0,0,3,1348099200,so much healthier,I think it is so important to give the best to...,NaN,False
15,262950,262951,B0000D1728,ANNHY2DW6MNFI,David,3,3,1,1295395200,Aspartame -- BEWARE,Forgot to add in the description that Wel Pac ...,1.0,False


In [4]:
###FIXING THE MISSING DATA IN THE SUMMARY COLUMN
data['newSummary'] = np.where(pd.isnull(data['Summary']) == True, 'nn', data['Summary'])
##GETTING A REPORT OF MISSING VALUES
data.isnull().sum()

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Unnamed: 0                    0
Id                            0
ProductId                     0
UserId                        0
ProfileName                   5
HelpfulnessNumerator          0
HelpfulnessDenominator        0
Score                         0
Time                          0
Summary                      22
Text                          0
helpScore                 66632
helpful                       0
newSummary                    0
dtype: int64

In [5]:
#features from Amazon.csv to add to feature set
#pandas creating new columns in the data frame

#length of the review
data['reviewLen'] = data['Text'].str.len()
#number of semi-colons
data['hasSC'] = data['Text'].str.count(';')
#number of exclaimation marks
data['hasEX'] = data['Text'].str.count('!')
#number of questions marks
data['hasQ'] = data['Text'].str.count('\?')
#total common punctuation count
data['punctCount'] = data['Text'].str.count('[.,!;:()/\?-]')
#ratio of punctuation to words
data['punctToWords'] = data['punctCount'] / data['reviewLen']
#average word length
data['avWordLength'] = data['Text'].str.len() // (data['Text'].str.count(' ') + 1)
#summary length
data['sumLen'] = data['newSummary'].str.count('\S')
#summary average word length
data['avSumWordLen'] = data['newSummary'].str.len() // (data['newSummary'].str.count(' ') + 1)
#summary exclaimation marks
data['sumHasEX'] = data['newSummary'].str.count('!')
#summary question marks
data['sumHasQ'] = data['newSummary'].str.count('\?')

data['date'] = pd.to_datetime(data['Time'],unit='s')
###getting day of the week (maybe Sundays are better times to write reviews)
data['day_of_week'] = data['date'].dt.dayofweek
#score is less than four
##data['scoreType'] = np.where(data['Score'] < 4, 1, 0)
data['scoreType'] = data['Score']
#data['newScore'] = np.where(data['HelpfulnessDenominator'] > 2 & data['HelpfulnessDenominator']/data['HelpfulnessNumerator'] > 0.65, 1, 0)
data['newScore'] = np.where(data['HelpfulnessNumerator']/data['HelpfulnessDenominator'] > 0.65, 1, 0)
data['newScore2'] = np.where(data['HelpfulnessDenominator'] > 2, 1, 0)
data['newScore3'] = np.where(data['newScore'] + data['newScore2'] == 2, 1, 0)

    
#wanted to get time of day but the timestamps only have dates...
#data['time_of_day'] = data['date'].dt.hour

#adjective to word ratio

print(data.shape)
data.head(5)


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

(164213, 31)


,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,...,sumLen,avSumWordLen,sumHasEX,sumHasQ,date,day_of_week,scoreType,newScore,newScore2,newScore3
0,138806,138807,B000E63LME,A1CQGW1AOD0LF2,"Alena K. ""Alena""",1,2,2,1294185600,Not as pictured.,...,14,5,0,0,2011-01-05,2,2,0,0,0
3,485307,485308,B001RVFERK,A25W349EE97NBK,Tangent4,1,1,4,1248307200,I wanted to love these...,...,21,5,0,0,2009-07-23,3,4,1,0,0
10,201170,201171,B00395570G,A2V2SL3JIOXYVM,doodle,0,0,1,1325203200,What happened 2 my once-favorite coffee?,...,35,6,0,1,2011-12-30,4,1,0,0,0
12,105099,105100,B0036VLZDQ,A1WR5TWGNODAHA,Katie R.,0,0,3,1348099200,so much healthier,...,15,5,0,0,2012-09-20,3,3,0,0,0
15,262950,262951,B0000D1728,ANNHY2DW6MNFI,David,3,3,1,1295395200,Aspartame -- BEWARE,...,17,6,0,0,2011-01-19,2,1,1,1,1


In [6]:


##pulling out values and making them vectors
XScore = data.iloc[:, 7].values.reshape(data.shape[0], 1)
XreviewLen = data.iloc[:, 14].values.reshape(data.shape[0], 1)
XhasSC = data.iloc[:, 15].values.reshape(data.shape[0], 1)
XhasEX = data.iloc[:, 16].values.reshape(data.shape[0], 1)
XhasQ = data.iloc[:, 17].values.reshape(data.shape[0], 1)
XpunctCount = data.iloc[:, 18].values.reshape(data.shape[0], 1)
XpunctToWords = data.iloc[:, 19].values.reshape(data.shape[0], 1)
XavWordLength = data.iloc[:, 20].values.reshape(data.shape[0], 1)
XsumLen = data.iloc[:, 21].values.reshape(data.shape[0], 1)
XavSumWordLen = data.iloc[:, 22].values.reshape(data.shape[0], 1)
XsumHasEX = data.iloc[:, 23].values.reshape(data.shape[0], 1)
XsumHasQ = data.iloc[:, 24].values.reshape(data.shape[0], 1)
#Xdate = data.iloc[:, 24].values.reshape(data.shape[0], 1)
Xday_of_week = data.iloc[:, 26].values.reshape(data.shape[0], 1)
XscoreType = data.iloc[:, 27].values.reshape(data.shape[0], 1)
#Xtoadd = np.concatenate((XScore, XreviewLen, XhasSC, XhasEX, XhasQ, XpunctCount, XpunctToWords, XavWordLength, Xday_of_week, XscoreType), axis=1)
Xtoadd = np.concatenate((XScore, XreviewLen, XhasSC, XhasEX, XhasQ, XpunctCount, XpunctToWords, XavWordLength, XsumLen, XavSumWordLen, XsumHasEX, XsumHasQ, Xday_of_week, XscoreType), axis=1)

In [7]:
# report on training and test sets
def print_results():
    print('Error rate on training set: ')
    print((y_train != y_pred).sum() / X_train.shape[0])
    print('Accuracy rate on training set: ')
    print(1 - (y_train != y_pred).sum() / X_train.shape[0])
    print('True positive rate on training tet:')
    print(((y_train==True) & (y_pred==True)).sum() / y_train.sum())
    print('**************')
    print('Error rate on test set: ')
    print((y_test != y_pred_test).sum() / X_test.shape[0])
    print('Accuracy rate on test set: ')
    print(1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    print('True positive rate on test set')
    print(((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    print('True negative rate on test set')
    print(((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))

In [8]:
# vectorize Bag of Words from review text; as sparse matrix
#from sklearn.feature_extraction.text import HashingVectorizer
#restricting to 2 to the power of 17 features
#hv = HashingVectorizer(n_features=2 ** 17, non_negative=True)
#X = hv.transform(data.Text)
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1, ngram_range=(2,3))
X = vectorizer.fit_transform(data.Text)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
#sumVectorizer = TfidfVectorizer(min_df=1, ngram_range=(1,1))
sumVectorizer = CountVectorizer(min_df=1, binary = True)
#sumVectorizer = CountVectorizer()
sumX = sumVectorizer.fit_transform(data.newSummary)
#vectorizer = CountVectorizer()
#X = vectorizer.fit_transform(data.Text)

In [10]:
# convert additional features to sparse matrix and concatenate onto the bag of words sparse matrix
from scipy.sparse import csr_matrix, hstack
XtoaddSparse = csr_matrix(Xtoadd)
Xfinal = hstack([X, sumX, XtoaddSparse])
#Xfinal = hstack([XtoaddSparse])
X = csr_matrix(Xfinal)

In [11]:
# size of feature set
print(X.shape)

(164213, 6469577)


In [12]:
# define y
y = data.iloc[:, 28].values
y.shape

(164213,)

In [13]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [14]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [15]:
# MODEL: SVM, linear defaults to Scaled Vector Machine
#incrementing each weight to reduce cost (defaults five passes)
# dealing with two vectors weights over and over
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results()

Error rate on training set: 
0.00385388302639
Accuracy rate on training set: 
0.996146116974
True positive rate on training tet:
0.994160030338
**************
Error rate on test set: 
0.293804806755
Accuracy rate on test set: 
0.706195193245
True positive rate on test set
0.626436439416
True negative rate on test set
0.747451415196


In [16]:
# MODEL: logistic regression
#log parameter, meaning logistic / tinker with alpha 
#here alpha is the regularization parameter  (default is 0.0001)
#here more penalizing the weights (hope to stop overfitting)
#need to specific the seed so that it doesn't totally randomize
#could loop through alphas and graph the accuracy rate
#could graphic the cost function and see when the number of iterations plateau
from sklearn import linear_model
#clf = linear_model.SGDClassifier(loss='log', n_iter=100, penalty='elasticnet',alpha=0.00001)
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results()

Error rate on training set: 
0.00260985306527
Accuracy rate on training set: 
0.997390146935
True positive rate on training tet:
0.995702186828
**************
Error rate on test set: 
0.2944746671
Accuracy rate on test set: 
0.7055253329
True positive rate on test set
0.625186067282
True negative rate on test set
0.747081831901


In [17]:
# MODEL: Naive Bayes
#this improved the true positives / maybe a different model for true positives
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results()

Error rate on training set: 
0.00290563641267
Accuracy rate on training set: 
0.997094363587
True positive rate on training tet:
0.998280874731
**************
Error rate on test set: 
0.27547499188
Accuracy rate on test set: 
0.72452500812
True positive rate on test set
0.573325394463
True negative rate on test set
0.802734916382


In [18]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results()

Error rate on training set: 
0.00362769576073
Accuracy rate on training set: 
0.996372304239
True positive rate on training tet:
0.997269624573
**************
Error rate on test set: 
0.293216141604
Accuracy rate on test set: 
0.706783858396
True positive rate on test set
0.635069961298
True negative rate on test set
0.743878776679
